# Семинар 8. Линейные методы. Vowpal Wabbit.

Vowpal Wabbit on GitHub: https://github.com/JohnLangford/vowpal_wabbit

Vowpal Wabbit Tutorial: https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_path = '../Contests/contest1_stackoverflow/train-contest.csv'

In [3]:
data = pd.read_csv(train_path)
data.head()

,PostId,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
0,11385967,07/08/2012 19:23:47,109676,04/27/2009 18:56:57,2382,81,Converting VB.NET Web Form to C# and wiring up...,"In My VB.NET web page, I have this standard ev...",c#,NaN,NaN,NaN,NaN,NaN,0
1,11351831,07/05/2012 20:09:03,1160891,01/20/2012 15:14:57,32,0,How to install visual studio 2010 in alternate...,"I'm trying to install VS 2010 on D drive, but ...",visual-studio-2010,installation,NaN,NaN,NaN,NaN,0
2,6411162,06/20/2011 12:30:47,706293,04/13/2011 15:02:59,21,2,Advise on resources where to learn IT Infrastr...,My IT knowledge resides mainly in SW developme...,documentation,tutorials,knowledge,NaN,NaN,06/20/2011 23:25:56,1
3,9962611,04/01/2012 07:16:03,168143,09/03/2009 20:31:24,3052,102,How is my single-threaded Rails app handling c...,I have a single-threaded Rails app running on ...,ruby-on-rails,ruby,concurrency,heroku,rack,NaN,0
4,11361230,07/06/2012 11:21:17,1047058,11/15/2011 07:34:27,1,0,"picturebox not showing anything in wndproc ""ho...",im trying to show printscreen image in picture...,c#,api,picturebox,NaN,NaN,NaN,0


In [4]:
print data.OpenStatus[10]
print data.BodyMarkdown[10]

0
I have a global variable called 'name', when a user clicks on a button a text boxes content is queried and 'name' is set to the contents then the name gets printed in a different jquery page.

My problem is name is never printed as the name it was set to, it's always printed as what it was initially set to instead.

Here's my callback function:

		<script type="text/javascript">
			$("#finalize").click(function(e)
			{
				g_name = "BLAH";
				return true;
			});
		</script>

For simplicity I've set the name to just be 'BLAH' on the click.

g_name is declared globally (i.e. outside the function scope).

In the other jquery page I have:

    document.write('<label for="testname">Name:</label>');
    				document.write('<input type="text" name="testname" id="testname" value="');
    				document.write(g_name)
    				document.write('"/>');

And g_name is always shown as something else, whatever g_name was initially initilized to.

The button whose click callback sets the global variabl

In [5]:
data_train = data.iloc[:50000, :]
data_test = data.iloc[70000:, :]

In [6]:
import re

def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            text = filter(lambda x: len(x) > 1, re.split("[^a-z]",
                                    row.BodyMarkdown.lower()))
            text = ' '.join(text)
            if row.OpenStatus == 1:
                target = 1
            else:
                target = -1
            fout.write('{0} |n 0:{1} {2} |t {3}\n'.format(target, 
                                        row.ReputationAtPostCreation,
                                        row.Tag1,
                                        text))

In [7]:
save_to_vw(data_train, 'train.vw')
save_to_vw(data_test, 'test.vw')

In [8]:
!head -n 2 train.vw

-1 |n 0:2382 c# |t in my vb net web page have this standard event note the handles clause on teh event declaration protected sub page load byval sender as object byval as system eventargs handles me load end sub in my web app have this protected void page load object sender system eventargs since doesn have handles equivalent and from what ve seen event handlers are wired up using delegate syntax was looking for this but could not foind it in the aspx page aspx cs file or the aspx designer cs file in vb would have two drop down lists at the top of the code editor window and could select any object on the web form or the web form itself and see the possible events for the object selecting the event would either take me to the event handler or if it didn exists it would create the stub for me know that the properties window in and think in vb too has an event tab that shows the list of events for the selected object gui object but page doesn appear as an object that can be selected where

In [9]:
!vw -d train.vw -c -k -f model.vw --passes 10 

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      254
0.763886   0.527772          2      2.0    -1.0000  -0.2735       44
1.590055   2.416223          4      4.0    -1.0000  -0.0876       82
1.230513   0.870972          8      8.0     1.0000  -0.2551       77
0.712284   0.194056         16     16.0    -1.0000  -0.6717      190
0.773382   0.834480         32     32.0     1.0000   0.0282       21
0.963355   1.153328         64     64.0     1.0000   0.4354      100
0.971244   0.979133        128    128.0     1.0000   0.1124       52
1.004249   1.037254        256    256.0    -1.0000   0.2181      279
0.961495   0.918740        512    

In [10]:
!vw -d test.vw -i model.vw -t -p pred.txt

only testing
predictions = pred.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.152298   1.152298          1      1.0    -1.0000   0.0735       63
1.548524   1.944750          2      2.0    -1.0000   0.3945       31
1.304907   1.061290          4      4.0     1.0000  -0.3451      103
0.919371   0.533836          8      8.0     1.0000   0.7522       33
0.708812   0.498253         16     16.0    -1.0000  -1.0000      139
0.694174   0.679535         32     32.0     1.0000   0.0276      154
0.757704   0.821234         64     64.0     1.0000  -0.3299       36
0.843794   0.929883        128    128.0     1.0000   1.0000      108
0.821948   0.800103        256    256.0     1.0000   0.4271       30
0.818219   0.814491        512    512.0     1.0000   0.5219       16
0.80

In [11]:
!head -n 10 pred.txt

0.073451
0.394543
-0.440162
-0.345051
1.000000
0.707419
-0.410080
0.752177
0.555422
0.082952


In [12]:
from sklearn.metrics import roc_auc_score

def calc_vw_qual():
    preds = pd.read_csv('pred.txt', header=None).iloc[:, 0].values
    target = data_test.OpenStatus.values
    print roc_auc_score(target, preds)

In [13]:
calc_vw_qual()

0.773175444484


In [18]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1
!vw -d test.vw -i model.vw -t -p pred.txt
print '\n\n\n'
calc_vw_qual()

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      254
0.824177   0.648353          2      2.0    -1.0000  -0.1948       44
1.239373   1.654570          4      4.0    -1.0000  -0.1296       82
1.074697   0.910021          8      8.0     1.0000  -0.2473       77
0.666821   0.258945         16     16.0    -1.0000  -0.5618      190
0.758021   0.849221         32     32.0     1.0000  -0.0742       21
0.889370   1.020718         64     64.0     1.0000   0.2461      100
0.948542   1.007714        128    128.0     1.0000  -0.0434       52
0.957862   0.967181        256    256.0    -1.0000   0.1023      279
0.921585   0.885309        512    

In [20]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2
!vw -d test.vw -i model.vw -t -p pred.txt
print '\n\n\n'
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      504
0.893775   0.787549          2      2.0    -1.0000  -0.1126       84
1.116160   1.338546          4      4.0    -1.0000  -0.0854      160
1.027996   0.939831          8      8.0     1.0000  -0.1864      150
0.708380   0.388764         16     16.0    -1.0000  -0.4080      376
0.781467   0.854553         32     32.0     1.0000  -0.0693       38
0.906742   1.032018         64     64.0     1.0000   0.1270      196
0.954535   1.002328        128    128.0     1.0000   0.0251      100
0.958956   0.963377        256    256.0    -1.0000  -0.0042      5

In [21]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 --skips t2
!vw -d test.vw -i model.vw -t -p pred.txt
print '\n\n\n'
calc_vw_qual()

Generating 2-grams for t namespaces.
Generating 2-skips for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000     1001
0.932438   0.864876          2      2.0    -1.0000  -0.0700      161
1.069380   1.206322          4      4.0    -1.0000  -0.0552      313
1.013547   0.957714          8      8.0     1.0000  -0.1455      293
0.764101   0.514655         16     16.0    -1.0000  -0.2995      745
0.818270   0.872439         32     32.0     1.0000  -0.0604       69
0.933228   1.048186         64     64.0     1.0000   0.0474      385
0.968953   1.004679        128    128.0     1.0000   0.0376      193
0.972765   0.976577        25

In [22]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 -b 28
!vw -d test.vw -i model.vw -t -p pred.txt
print '\n\n\n'
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      504
0.893775   0.787549          2      2.0    -1.0000  -0.1126       84
1.116160   1.338546          4      4.0    -1.0000  -0.0854      160
1.027811   0.939461          8      8.0     1.0000  -0.1863      150
0.709204   0.390598         16     16.0    -1.0000  -0.4044      376
0.781565   0.853925         32     32.0     1.0000  -0.0694       38
0.907234   1.032904         64     64.0     1.0000   0.1415      196
0.956017   1.004799        128    128.0     1.0000   0.0238      100
0.958802   0.961587        256    256.0    -1.0000   0.0139      5